In [1]:
import qutip as qp
import numpy as np
import matplotlib.pyplot as plt

In [11]:
qstate = [qp.basis(4,i) for i in range(4)]
sig = [[qstate[i] * qstate[j].dag() for j in range(4)] for i in range(4)]
up = (sig[0][1] + sig[1][0]) / 2
us = (sig[1][2] + sig[2][1]) / 2
psi0 = qp.ket2dm(qstate[0])
target_state = sig[2][2]

In [42]:
Delta, delp = 0.0, 0.0
H0 = Delta * sig[1][1] + delp * sig[2][2]

T = 1
gamma = 5.0
n_steps = 30
Omega_max = 20

tlist = np.linspace(0, T, n_steps + 1)
delt = tlist[1] - tlist[0]

opts = qp.Options(atol=1e-11, rtol=1e-9, nsteps=int(1e6))
opts.normalize_output = False

In [43]:
#def apply_superoperator(L, ρ):
#    return vector_to_operator(L * operator_to_vector(ρ))

#def _qstep(action, qstate):
#    H = H0 + action[0] * up + action[1] * us
#    L = (qp.liouvillian(H, [np.sqrt(gamma) * sig[3][1]]) * delt).expm()
#    return qp.apply_superoperator(L, qstate)

def vals2amps(vals):
    message = "vals must be 1-D array with shape (n,) where n is even"
    assert len(vals.shape) == 1, message
    assert vals.shape[0] % 2 == 0, message
    return vals.reshape(-1, 2, order="F")

def function_from_array(y, x):
    """Return function given an array and time points."""

    if y.shape[0] != x.shape[0]:
        raise ValueError("y and x must have the same first dimension")

    yx = np.column_stack((y, x))
    yx = yx[yx[:, -1].argsort()]

    def func(t, args):
        idx = np.searchsorted(yx[1:, -1], t, side="right")
        return yx[idx, 0]

    return func

def vals2amps(vals):
    message = "vals must be 1-D array with shape (n,) where n is even"
    assert len(vals.shape) == 1, message
    assert vals.shape[0] % 2 == 0, message
    return vals.reshape(-1, 2, order="F")

def run_mesolvevolution(amps):
    Omega_p = amps[:, 0]
    Omega_s = amps[:, 1]

    fp = function_from_array(Omega_p, tlist[:-1])
    fs = function_from_array(Omega_s, tlist[:-1])

    H = [H0, [up, fp], [us, fs]]

    result = qp.mesolve(
        H,
        psi0,
        tlist,
        c_ops=[np.sqrt(gamma) * sig[3][1]],
        options=opts,
    )
    return result

In [44]:
np.random.seed(11235813)

init_vals = np.random.rand(2 * n_steps) * Omega_max

output = run_mesolvevolution(vals2amps(init_vals))

In [47]:
qp.expect(target_state, output.states[-1])

0.13842597182944466

In [49]:
import torch as t

In [67]:
init_vals_tensor = t.rand(2 * n_steps) * Omega_max

output = run_mesolvevolution(init_vals_tensor.view(-1,2))

In [68]:
qp.expect(target_state, output.states[-1])

0.00644287483668893

In [94]:
def apply_superoperator(L, ρ):
    return qp.vector_to_operator(L * qp.operator_to_vector(ρ))

def _qstep(action, qstate):
    H = H0 + action[0] * up + action[1] * us
    L = (qp.liouvillian(H, [np.sqrt(gamma) * sig[3][1]]) * delt).expm()
    print(help(L))
    return apply_superoperator(L, qstate)

def run_qstepevolution(amps):
        Omega_p = amps[:, 0]
        Omega_s = amps[:, 1]

        states = [psi0]

        for i in range(n_steps):
            states.append(_qstep([Omega_p[i], Omega_s[i]], states[-1]))

        return states

In [95]:
output = run_qstepevolution(vals2amps(init_vals))

Help on Qobj in module qutip.qobj object:

class Qobj(builtins.object)
 |  Qobj(inpt=None, dims=None, shape=None, type=None, isherm=None, copy=True, fast=False, superrep=None, isunitary=None)
 |  
 |  A class for representing quantum objects, such as quantum operators
 |  and states.
 |  
 |  The Qobj class is the QuTiP representation of quantum operators and state
 |  vectors. This class also implements math operations +,-,* between Qobj
 |  instances (and / by a C-number), as well as a collection of common
 |  operator/state operations.  The Qobj constructor optionally takes a
 |  dimension ``list`` and/or shape ``list`` as arguments.
 |  
 |  Parameters
 |  ----------
 |  inpt : array_like
 |      Data for vector/matrix representation of the quantum object.
 |  dims : list
 |      Dimensions of object used for tensor products.
 |  shape : list
 |      Shape of underlying data structure (matrix shape).
 |  copy : bool
 |      Flag specifying whether Qobj should get a copy of the
 |  

Help on Qobj in module qutip.qobj object:

class Qobj(builtins.object)
 |  Qobj(inpt=None, dims=None, shape=None, type=None, isherm=None, copy=True, fast=False, superrep=None, isunitary=None)
 |  
 |  A class for representing quantum objects, such as quantum operators
 |  and states.
 |  
 |  The Qobj class is the QuTiP representation of quantum operators and state
 |  vectors. This class also implements math operations +,-,* between Qobj
 |  instances (and / by a C-number), as well as a collection of common
 |  operator/state operations.  The Qobj constructor optionally takes a
 |  dimension ``list`` and/or shape ``list`` as arguments.
 |  
 |  Parameters
 |  ----------
 |  inpt : array_like
 |      Data for vector/matrix representation of the quantum object.
 |  dims : list
 |      Dimensions of object used for tensor products.
 |  shape : list
 |      Shape of underlying data structure (matrix shape).
 |  copy : bool
 |      Flag specifying whether Qobj should get a copy of the
 |  

Help on Qobj in module qutip.qobj object:

class Qobj(builtins.object)
 |  Qobj(inpt=None, dims=None, shape=None, type=None, isherm=None, copy=True, fast=False, superrep=None, isunitary=None)
 |  
 |  A class for representing quantum objects, such as quantum operators
 |  and states.
 |  
 |  The Qobj class is the QuTiP representation of quantum operators and state
 |  vectors. This class also implements math operations +,-,* between Qobj
 |  instances (and / by a C-number), as well as a collection of common
 |  operator/state operations.  The Qobj constructor optionally takes a
 |  dimension ``list`` and/or shape ``list`` as arguments.
 |  
 |  Parameters
 |  ----------
 |  inpt : array_like
 |      Data for vector/matrix representation of the quantum object.
 |  dims : list
 |      Dimensions of object used for tensor products.
 |  shape : list
 |      Shape of underlying data structure (matrix shape).
 |  copy : bool
 |      Flag specifying whether Qobj should get a copy of the
 |  

In [84]:
output[-1]

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.366443  +0.j          0.        +0.06222454j -0.22522263+0.j
   0.        +0.j        ]
 [ 0.        -0.06222454j  0.01056616+0.j          0.        +0.03824435j
   0.        +0.j        ]
 [-0.22522263+0.j          0.        -0.03824435j  0.13842598+0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.48456486+0.j        ]]